# Updatable Nearest Neighbor Classifier


This notebook demonstrates the process of creating an updatable empty k-nearest neighbor model using coremltools.

In [22]:
number_of_dimensions = 128

from coremltools.models.nearest_neighbors import KNearestNeighborsClassifierBuilder
builder = KNearestNeighborsClassifierBuilder(input_name='input',
                                             output_name='output',
                                             number_of_dimensions=number_of_dimensions,
                                             default_class_label='defaultLabel',
                                             number_of_neighbors=3,
                                             weighting_scheme='inverse_distance',
                                             index_type='linear')

builder.author = 'Core ML Tools Example'
builder.license = 'MIT'
builder.description = 'Classifies {} dimension vector based on 3 nearest neighbors'.format(number_of_dimensions)

builder.spec.description.input[0].shortDescription = 'Input vector to classify'
builder.spec.description.output[0].shortDescription = 'Predicted label. Defaults to \'defaultLabel\''
builder.spec.description.output[1].shortDescription = 'Probabilities / score for each possible label.'

builder.spec.description.trainingInput[0].shortDescription = 'Example input vector'
builder.spec.description.trainingInput[1].shortDescription = 'Associated true label of each example vector'

In [13]:
# By default an empty knn model is updatable
builder.is_updatable

True

In [14]:
# Let's confirm the number of dimension is set correctly
builder.number_of_dimensions

128

In [15]:
# Let's check what the value of 'numberOfNeighbors' is
builder.number_of_neighbors

3

In [16]:
# The number of neighbors is bounded by the default range...
builder.number_of_neighbors_allowed_range()

(1, 1000)

In [17]:
# If we try to set the number of neighbors to a value outside of this range
builder.number_of_neighbors = 1001

ValueError: number_of_neighbors is not within range bounds

In [18]:
# Instead of a range, you can a set individual values that are valid for the numberOfNeighbors parameter.
builder.set_number_of_neighbors_with_bounds(3, allowed_set={ 1, 3, 5 })

In [19]:
# Check out the results of the previous operation
builder.number_of_neighbors_allowed_set()

{1, 3, 5}

In [20]:
# And now if you attempt to set it to an invalid value...
builder.number_of_neighbors = 4

ValueError: number_of_neighbors is not valid

In [21]:
# And of course you can go back to a valid range
builder.set_number_of_neighbors_with_bounds(3, allowed_range=(1, 30))

In [17]:
# Let's see what the index type is
builder.index_type

'linear'

In [18]:
# Let's set the index to kd_tree with leaf size of 30
builder.set_index_type('kd_tree', 30)
builder.index_type

'kd_tree'

In [19]:
mlmodel_updatable_path = './UpdatableKNN.mlmodel'

# Save the updated spec
from coremltools.models import MLModel
mlmodel_updatable = MLModel(builder.spec)
mlmodel_updatable.save(mlmodel_updatable_path)